
#Install the libraries

In [ ]:
!pip install tf_slim
!pip install roboflow
!pip install tensorflow-io
!pip install tf-models-official==2.8.0
!pip install avro-python3 
!pip install tf_slim==1.1.0
!pip install lvis
!pip install tensorflow_io==0.23.1
!pip install keras==2.7.0
!pip install opencv-python-headless==4.5.2.52

import tensorflow_io as tfio
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import tensorflow as tf
import sys
import pickle
import zipfile
import shutil

from roboflow import Roboflow

# Upload images from Roboflow

In [ ]:
rf = Roboflow(api_key="your-api_key")
project = rf.workspace("workspace-dir-roboflow").project("your-project-name-roboflow")
dataset = project.version(8).download("tensorflow")

# Install the Object Detection library





In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
#!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

# Conversion of Roboflow CSV files to TFrecord

In [ ]:
records_fname = ['valid', 'train']

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

# TO-DO replace this with label map
# for multiple labels add more else if statements
def class_text_to_int(row_label):
    if row_label == 'cultivo':  # 'ship':
        return 1
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example1(group, path):
     ruta = os.path.join(path, '{}'.format(group.filename))
     print(ruta)

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'png'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
for record in records_fname:
  output_path = record+".record"
  image_dir = dataset.location+"/"+record+"/"
  csv_input = image_dir+"_annotations.csv"

  writer = tf.io.TFRecordWriter(output_path)
  path = os.path.join(image_dir)
  examples = pd.read_csv(csv_input)
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

  writer.close()
  output_path = os.path.join(os.getcwd(), output_path)
  print('Successfully created the TFRecords: {}'.format(output_path))

# Loading Tensorflow 2 Detection Model Zoo Models

In [ ]:
# the labelmap file is created, which allows us to obtain the classes for detection
%cd /content

labels = [{'name': 'cultivo', 'id': 1}]
with open("label_map.pbtxt", "w") as f:
  for label in labels:
    f.write('item {\n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

#### SSD MobileNet V2 FPNLite

In [ ]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz \
-O /content/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
output_path_mobilenet = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
output_path_mobilenet = os.path.join(os.getcwd(), output_path_mobilenet)
print("The folder is stored in {}".format(output_path_mobilenet))

### EfficientDet D1

In [ ]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz \
-O /content/efficientdet_d1_coco17_tpu-32.tar.gz

In [ ]:
!tar -zxvf /content/efficientdet_d1_coco17_tpu-32.tar.gz
output_path_efficientdet = 'efficientdet_d1_coco17_tpu-32'
output_path_efficientdet = os.path.join(os.getcwd(), output_path_efficientdet)
print("The folder is stored in {}".format(output_path_efficientdet))

### SSD ResNet152 V1 FPN (RetinaNet152)

In [ ]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz \
-O /content/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
!tar -zxvf /content/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz
output_path_resnet = 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8'
output_path_resnet = os.path.join(os.getcwd(), output_path_resnet)
print("The folder is stored in {}".format(output_path_resnet))

# Configuration of the pipeling.config file for each loaded model

In [ ]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
path_training = 'path-load-models'
os.mkdir(path_training)

In [ ]:
label_map_pbtxt_fname = "path-labelmap-file"
train_record_fname = "path-train-record-file"
test_record_fname = "path-valid-record-file"

#### Configuration SSD MobileNet

In [ ]:
os.mkdir(path_training+"/mobilenet")
source_config_mobilenet = "{}/pipeline.config".format(output_path_mobilenet)
target_config_mobilenet = "{}/pipeline.config".format(path_training+"/mobilenet")
shutil.copy(source_config_mobilenet, target_config_mobilenet)

'/content/modelos/mobilenet/pipeline.config'

In [ ]:
config_mobilenet = config_util.get_configs_from_pipeline_file(target_config_mobilenet)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config_mobilenet, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config

In [ ]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = "{}/checkpoint/ckpt-0".format(output_path_mobilenet)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [ ]:
pipeline_config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config_mobilenet, "wb") as f:
  f.write(config_text)

#### Configuration EfficientDet

In [ ]:
os.mkdir(path_training+"/efficientdet")
source_config_efficientdet = "{}/pipeline.config".format(output_path_efficientdet)
target_config_efficientdet = "{}/pipeline.config".format(path_training+"/efficientdet")
shutil.copy(source_config_efficientdet, target_config_efficientdet)

In [ ]:
config_efficientdet = config_util.get_configs_from_pipeline_file(target_config_efficientdet)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config_efficientdet, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config

In [ ]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = "{}/checkpoint/ckpt-0".format(output_path_efficientdet)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [ ]:
pipeline_config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config_efficientdet, "wb") as f:
  f.write(config_text)

#### Configuration SSD ResNet152

In [ ]:
os.mkdir(path_training+"/resnet")
source_config_resnet = "{}/pipeline.config".format(output_path_resnet)
target_config_resnet = "{}/pipeline.config".format(path_training+"/resnet")
shutil.copy(source_config_resnet, target_config_resnet)

In [ ]:
config_resnet = config_util.get_configs_from_pipeline_file(target_config_resnet)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config_resnet, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config

In [ ]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = "{}/checkpoint/ckpt-0".format(output_path_resnet)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [ ]:
pipeline_config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config_resnet, "wb") as f:
  f.write(config_text)

# Model training 

In [ ]:
path_training = '/content/entrenamientos'
os.mkdir(path_training)

##### SSD MobileNet

In [ ]:
model_dir_mobilenet = path_training+"/mobilenet"
num_step = 20000

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config_mobilenet} \
--model_dir={model_dir_mobilenet} \
--num_train_steps={num_step} \
--eval_on_train_data=True


In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/entrenamientos/mobilenet'

In [ ]:
output_directory_ssdmobilenet = "/content/fine_tuned_mobilenet"

!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config_mobilenet} \
--trained_checkpoint_dir {model_dir_mobilenet} \
--output_directory {output_directory_ssdmobilenet}

In [ ]:
!zip -r /content/fine_tuned_mobilenet.zip /content/fine_tuned_mobilenet

##### EfficientDet

In [ ]:
model_dir_efficientdet = path_training+"/efficientdet"
os.mkdir(model_dir_efficientdet)

num_step = 20000

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config_efficientdet} \
--model_dir={model_dir_efficientdet} \
--num_train_steps={num_step} \
--eval_on_train_data=True



In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/entrenamientos/efficientdet'

In [ ]:
output_directory_efficientdet = "/content/fine_tuned_efficientdet"

!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config_efficientdet} \
--trained_checkpoint_dir {model_dir_efficientdet} \
--output_directory {output_directory_efficientdet}

In [ ]:
!zip -r /content/fine_tuned_efficientdet.zip /content/fine_tuned_efficientdet

#### SSD ResNet152

In [ ]:
model_dir_resnet = path_training+"/resnet"
os.mkdir(model_dir_resnet)

num_step = 20000

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config_resnet} \
--model_dir={model_dir_resnet} \
--num_train_steps={num_step} \
--eval_on_train_data=True

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/entrenamientos/resnet'

In [ ]:
output_directory_resnet = "/content/fine_tuned_resnet"

!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config_resnet} \
--trained_checkpoint_dir {model_dir_resnet} \
--output_directory {output_directory_resnet}

In [ ]:
!zip -r /content/fine_tuned_resnet.zip /content/fine_tuned_resnet